# SPCS SFTP Connectivity Test

This notebook provides modular components for testing SFTP connectivity from Snowflake Container Services (SPCS):

## Quick Start Options:
1. **Basic Test**: Configure parameters and run network/auth tests
2. **Full Setup**: Use cells to set up PyPI access and SFTP EAI as needed
3. **Custom Setup**: Pick and choose individual components as needed

## Components Available:
- **Configuration**: Set your SFTP connection parameters
- **PyPI Setup**: Optional - enables installing SFTP driver from PyPI
- **Network Test**: Tests basic network connectivity to SFTP host
- **Authentication Test**: Tests SFTP authentication and list directories
- **SFTP EAI**: Optional - configures External Access Integration for SFTP


**Note: You must restart the Session after making EAI changes for them to take effect**

In [ ]:
# SFTP Connectivity Test Configuration
# Update these parameters with your actual SFTP connection details

# User Configuration - UPDATE THIS
SFTP_HOST = "s-abcdef123.server.us-east-2.rds.amazonaws.com"
SFTP_PORT = 22
SFTP_USER = "demo"
SFTP_PASS = "password"

print(f"Configuration:")
print(f"SFTP Host: {SFTP_HOST}")
print(f"SFTP Port: {SFTP_PORT}")
print(f"SFTP User: {SFTP_USER}")
print("\nReady to test connectivity...")

## 1. PyPI Setup (Optional)

Run these cells if you need to install paramiko from PyPI. This creates the necessary network rules and External Access Integration for PyPI access.

**Skip this section if you already have installed or have PyPI access configured.**


In [ ]:
-- Create Network Rule and External Access Integration for PyPI
-- Run this cell to enable installing Python packages from PyPI

CREATE OR REPLACE NETWORK RULE pypi_network_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('pypi.org', 'pypi.python.org', 'pythonhosted.org', 'files.pythonhosted.org');

CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION pypi_access_integration
  ALLOWED_NETWORK_RULES = (pypi_network_rule)
  ENABLED = true
  COMMENT = 'External Access Integration for PyPI package installation';

-- Grant usage on the integration
GRANT USAGE ON INTEGRATION pypi_access_integration TO ROLE ACCOUNTADMIN;

SHOW EXTERNAL ACCESS INTEGRATIONS LIKE 'pypi_access_integration';


In [ ]:
-- Apply PyPI integration to this notebook
-- Run this after creating the PyPI integration above

ALTER NOTEBOOK EAI_SFTP
  SET EXTERNAL_ACCESS_INTEGRATIONS = ('pypi_access_integration');


**NOTE: You will need to restart the Notebook session after applying an EAI**

In [ ]:
# Install paramiko
# Make sure PyPI access is configured first if you get connection errors
# You can run this cell twice; the first to install the driver, the second to confirm it is imported

try:
    import paramiko
    print("✅ paramiko already available")
except ImportError:
    print("📦 Installing paramiko...")
    %pip install paramiko
    print("✅ paramiko installed")

## 2. Connectivity Tests

These cells test connectivity to your SFTP host. Run them in order to diagnose any connection issues.

**Note: If you need to install paramiko, make sure to run the PyPI setup section first and restart your session.**


In [ ]:
# Network Connectivity Test
# Tests basic network connectivity to SFTP host and port

import socket

def test_network_connectivity(host, port, description):
    """Test network connectivity to SFTP host and port"""
    try:
        print(f"🔍 Testing {description}: {host}:{port}")

        # Test network connectivity
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(10)
        result = sock.connect_ex((host, port))
        sock.close()

        if result == 0:
            print(f"  ✅ Network connection successful")
            return True
        else:
            print(f"  ❌ Network connection failed (error code: {result})")
            print(f"  💡 This may indicate the need for a SFTP EAI")
            return False

    except Exception as e:
        print(f"  ❌ Network Error: {e}")
        return False

# Run network connectivity test
print("=" * 50)
print("NETWORK CONNECTIVITY TEST")
print("=" * 50)

network_result = test_network_connectivity(SFTP_HOST, SFTP_PORT, "SFTP Network Connectivity")

if network_result:
    print(f"\n✅ Network connectivity PASSED - SFTP host is reachable")
    print("You can proceed to test SFTP authentication.")
else:
    print(f"\n❌ Network connectivity FAILED")
    print("You need to configure a SFTP External Access Integration (EAI).")
    print("See the SFTP EAI Setup section below.")

In [ ]:
def test_sftp_connection_to_server(host, port, username, password):
    """Test SFTP connection to a specific server with detailed debugging."""
    print(f"\n{'='*60}")
    print(f"Host: {host}:{port}")
    print(f"Username: {username}")
    print(f"{'='*60}")
    
    try:
        # Enable paramiko debug logging (comment out for cleaner output)
        # logging.basicConfig(level=logging.DEBUG)
        # paramiko.util.log_to_file('paramiko_debug.log')
        
        print(f"Attempting to connect to {host}...")
        
        # Create SSH client
        ssh_client = paramiko.SSHClient()
        ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        
        # Connect to SSH server with detailed error handling
        print("Creating SSH connection...")
        ssh_client.connect(hostname=host,
                          port=port,
                          username=username,
                          password=password,
                          timeout=30,
                          look_for_keys=False,
                          allow_agent=False)
        
        print("✓ SSH connection and authentication successful.")
        
        # Create SFTP client from SSH connection
        print("Opening SFTP session...")
        sftp_client = ssh_client.open_sftp()
        
        try:
            # --- Connectivity Test: Directory Listing ---
            print("Performing directory listing...")
            remote_file_list = sftp_client.listdir()
            
            print("✓ Directory listing successful. Contents:")
            if not remote_file_list:
                print("  (Directory is empty)")
            else:
                for item in remote_file_list:
                    print(f"  - {item}")
            
            print(f"\n✓ SUCCESS: {host} connectivity test passed.")
            return True
            
        finally:
            # Close SFTP and SSH connections
            sftp_client.close()
            ssh_client.close()
            print("✓ Connections closed.")

    except paramiko.AuthenticationException as e:
        print(f"✗ FAILURE: Authentication failed for {host}")
        print(f"  Error Details: {e}")
        return False
    except paramiko.SSHException as e:
        print(f"✗ FAILURE: SSH connection error for {host}")
        print(f"  Error Details: {e}")
        return False
    except socket.error as e:
        print(f"✗ FAILURE: Could not establish connection to {host}")
        print(f"  Error Details: {e}")
        return False
    except Exception as e:
        print(f"✗ FAILURE: Unexpected error for {host}")
        print(f"  Error Type: {type(e).__name__}")
        print(f"  Error Details: {e}")
        return False


# --- Host Key Handling ---
# WARNING: Disabling host key checking is insecure and vulnerable to
# Man-in-the-Middle (MITM) attacks. This should ONLY be used for initial
# connectivity testing in a non-production environment.
# See Section 5 for production-safe host key verification methods.
print("WARNING: Host key verification is disabled. This is NOT secure for production.")

# Test primary server first
success = test_sftp_connection_to_server(
    SFTP_HOST, SFTP_PORT, SFTP_USER, SFTP_PASS
)

# If primary server fails, try alternative server         
if success:
    print(f"\n{'='*60}")
    print("✓ SFTP connectivity test completed successfully!")
    print("="*60)
else:
    print(f"\n{'='*60}")
    print("✗ SFTP connectivity tests failed.")
    print("Check the debug log file 'paramiko_debug.log' for detailed information.")
    print("="*60)


## 3. SFTP EAI Setup (Optional)

Run these cells if connectivity tests failed. This creates the necessary network rules and External Access Integration specifically for SFTP access.

**Skip this section if your connectivity tests passed.**

In [ ]:
-- Create Network Rule for SFTP connectivity
-- This allows egress traffic to your specific FTP host and port

-- Network rule for your SFTP database server
-- Uses the variables set in the configuration cell above
CREATE OR REPLACE NETWORK RULE SFTP_RULE
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('{{SFTP_HOST}}:{{SFTP_PORT}}');

-- Show the created network rule
DESCRIBE NETWORK RULE SFTP_RULE;

In [ ]:
-- Create External Access Integration for SFTP
-- This combines all the network rules into a single integration

CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION SFTP_EAI
  ALLOWED_NETWORK_RULES = (
    SFTP_RULE
  )
  ENABLED = TRUE
  COMMENT = 'External Access Integration for SFTP connectivity';

-- Grant usage on the integration to your role
GRANT USAGE ON INTEGRATION SFTP_EAI TO ROLE ACCOUNTADMIN;

SHOW EXTERNAL ACCESS INTEGRATIONS LIKE 'SFTP_EAI';

In [ ]:
-- Apply SFTP EAI to this notebook
-- This enables the notebook to access your SFTP Server
-- Include pypi_access_integration if you created it earlier

ALTER NOTEBOOK EAI_SFTP
  SET EXTERNAL_ACCESS_INTEGRATIONS = ('SFTP_EAI', 'pypi_access_integration');

-- Alternative: If you only need SFTP access (no PyPI), use:
-- ALTER NOTEBOOK SFTP
--   SET EXTERNAL_ACCESS_INTEGRATIONS = ('SFTP_EAI');